### Basic FPS and Drift Check

In [1]:
#!/usr/bin/env python3
# subscriber.py
import cv2, zmq, numpy as np, time
from collections import deque, defaultdict

# ---------- Config ----------
ZMQ_ADDR = "tcp://localhost:5555"
SUB_TOPICS = [b"kreo1", b"kreo2"]
FPS_WINDOW = 1.0        # seconds for fps moving window

# ---------- Helpers ----------
def fmt_ts(ts):
    # human-friendly timestamp with ms
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ts)) + f".{int((ts%1)*1000):03d}"

# ---------- ZMQ subscriber ----------
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect(ZMQ_ADDR)
sub.setsockopt(zmq.RCVHWM, 1)
sub.setsockopt(zmq.CONFLATE, 1)  # keep only last message

# ---- ACTIVE FLUSH ----
flushed = 0
while True:
    try:
        sub.recv_multipart(flags=zmq.NOBLOCK)
        flushed += 1
    except zmq.Again:
        break
if flushed > 0:
    print(f"[Subscriber] Flushed {flushed} stale messages.")
for t in SUB_TOPICS:
    sub.setsockopt(zmq.SUBSCRIBE, t)

# per-camera state
frames = {}
fps_windows = defaultdict(lambda: deque())   # deque of capture times

def update_fps(camera, capture_ts_corrected):
    dq = fps_windows[camera]
    now = time.time()
    dq.append(capture_ts_corrected)
    # pop older than window
    while dq and (capture_ts_corrected - dq[0]) > FPS_WINDOW:
        dq.popleft()
    fps = len(dq) / FPS_WINDOW
    return fps

def recv_latest(sub):
    msg = None
    while True:
        try:
            msg = sub.recv_multipart(flags=zmq.NOBLOCK)
        except zmq.Again:
            break
    return msg

print("[Subscriber] connected, waiting for frames... (Press ESC to exit)")

try:
    while True:
        parts = recv_latest(sub)
        if parts is None:
            continue

        # unpack message
        topic = parts[0]
        cam = topic.decode()

        if len(parts) >= 3:
            ts_part = parts[1]
            jpg_part = parts[2]
        else:
            ts_part = None
            jpg_part = parts[1]

        recv_time = time.time()

        try:
            cam_ts = float(ts_part.decode()) if ts_part else recv_time
        except:
            cam_ts = recv_time

        corrected_ts = cam_ts  # no drift correction

        img = cv2.imdecode(np.frombuffer(jpg_part, np.uint8), cv2.IMREAD_COLOR)
        if img is None:
            continue

        fps = update_fps(cam, corrected_ts)

        frames[cam] = {
            "img": img,
            "cam_ts": cam_ts,
            "corrected_ts": corrected_ts,
            "fps": fps,
        }

        # If we have at least 2 cameras, compute drift and show tiled view
        if all(k in frames for k in [t.decode() for t in SUB_TOPICS]):
            cams = [t.decode() for t in SUB_TOPICS]
            left = frames[cams[0]]
            right = frames[cams[1]]

            # compute drift in ms between corrected timestamps
            drift_s = abs(left["corrected_ts"] - right["corrected_ts"])
            drift_ms = drift_s * 1000.0

            # overlay text on each image
            def overlay(m):
                im = m["img"].copy()
                y = 20
                cv2.putText(im, f"{m['cam'] if 'cam' in m else ''}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (199,32,20), 2)
                cv2.putText(im, f"FPS: {m['fps']:.1f}", (10, y+26), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (14,117,5), 2)
                cv2.putText(im, f"cam_ts: {m['cam_ts']:.3f}", (10, y+52), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (5,12,117), 1)
                cv2.putText(im, f"corrected: {fmt_ts(m['corrected_ts'])}", (10, y+74), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (117,5,99), 1)
                return im

            # attach camera name in the dict for overlay convenience
            left["cam"] = cams[0]; right["cam"] = cams[1]
            left_im = overlay(left)
            right_im = overlay(right)

            # resize to same height and tile horizontally
            h = max(left_im.shape[0], right_im.shape[0])
            w1 = left_im.shape[1]
            right_resized = cv2.resize(right_im, (w1, h))
            tile = np.hstack([left_im, right_resized])

            # Draw drift and timestamp summary on top-left of tiled image
            cv2.putText(tile, f"Drift: {drift_ms:.1f} ms", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
            cv2.putText(tile, f"Host now: {fmt_ts(time.time())}", (10, 44), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

            cv2.imshow("Both Cameras (tiled)", tile)

        if cv2.waitKey(1) & 0xFF == 27:
            break

except KeyboardInterrupt:
    pass
finally:
    cv2.destroyAllWindows()
    sub.close()
    ctx.term()


[Subscriber] connected, waiting for frames... (Press ESC to exit)
